In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [ ]:
from google.colab import files
uploaded = files.upload()  # Select your ZIP file


Saving processed_fixed.zip to processed_fixed.zip


In [ ]:
import zipfile
import os

zip_path = list(uploaded.keys())[0]  # gets uploaded filename
extract_dir = "data"  # folder where it will be extracted

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

data_dir = os.path.join(extract_dir, "processed_fixed")
print(f"Dataset path: {data_dir}")
print("Sample files:", os.listdir(data_dir)[:5])


Dataset path: data/processed_fixed
Sample files: ['IMG_207_img.npy', 'IMG_283_den.npy', 'IMG_219_den.npy', 'IMG_79_img.npy', 'IMG_54_img.npy']


In [ ]:
# ---------------- Step 3: Imports ----------------
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
from torchvision import models

# ---------------- Step 4: Dataset ----------------
class CrowdDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.img_files = sorted([f for f in os.listdir(data_dir) if f.endswith("_img.npy")])

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_file = self.img_files[idx]
        den_file = img_file.replace("_img.npy", "_den.npy")
        img = np.load(os.path.join(self.data_dir, img_file)).astype(np.float32) / 255.0
        den = np.load(os.path.join(self.data_dir, den_file)).astype(np.float32)
        # Ensure channel-first for PyTorch
        if img.shape[0] != 3:
            img = np.transpose(img, (2,0,1))
        return torch.from_numpy(img), torch.from_numpy(den)

dataset = CrowdDataset(data_dir)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
print(f"Total samples: {len(dataset)}")


Total samples: 300


In [ ]:
# ---------------- Step 5: CSRNet Model ----------------
import torch.nn.functional as F

class CSRNet(nn.Module):
    def __init__(self, load_weights=False):
        super(CSRNet, self).__init__()
        vgg = models.vgg16(pretrained=load_weights)
        self.frontend = nn.Sequential(*list(vgg.features.children())[:23])
        self.backend = nn.Sequential(
            nn.Conv2d(512, 512, 3, padding=1, dilation=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=2, dilation=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=2, dilation=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 256, 3, padding=2, dilation=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 3, padding=2, dilation=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, 3, padding=2, dilation=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 1, 1)
        )

    def forward(self, x):
        x = self.frontend(x)
        x = self.backend(x)
        return x

# ---------------- Step 6: Device ----------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = CSRNet().to(device)
criterion = nn.MSELoss(reduction='sum')  # standard for crowd counting
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)


Using device: cuda


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
# ---------------- Step 7: Training Loop with Proper Loss Calculation ----------------
num_epochs = 100
checkpoint_dir = "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

for epoch in range(num_epochs):

    model.train()
    epoch_loss = 0.0
    epoch_count_err = 0.0
    total_batches = 0

    for imgs, dens in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):

        imgs = imgs.to(device)
        dens = dens.to(device)

        # Predict
        outputs = model(imgs)

        # Batch loss
        loss = criterion(outputs, dens)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate batch loss
        epoch_loss += loss.item()
        total_batches += 1

        # -------- Count Error per batch --------
        # sum() gives total predicted count
        pred_count = outputs.detach().sum().item()
        gt_count = dens.detach().sum().item()

        batch_count_err = abs(pred_count - gt_count)
        epoch_count_err += batch_count_err

    # -------- Epoch Loss --------
    avg_epoch_loss = epoch_loss / total_batches
    avg_epoch_count_err = epoch_count_err / total_batches

    print(f"\n🔵 Epoch {epoch+1}/{num_epochs}")
    print(f"   Avg Epoch Loss      : {avg_epoch_loss:.6f}")
    print(f"   Avg Count Error     : {avg_epoch_count_err:.2f}")

    # ---------------- Save Checkpoint ----------------
    checkpoint_path = os.path.join(checkpoint_dir, f"csrnet_epoch{epoch+1}.pth")
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': avg_epoch_loss
    }, checkpoint_path)

    print(f"   ✔ Checkpoint saved at: {checkpoint_path}")


Epoch 1/100: 100%|██████████| 300/300 [00:40<00:00,  7.42it/s]



🔵 Epoch 1/100
   Avg Epoch Loss      : 4.963520
   Avg Count Error     : 73.75
   ✔ Checkpoint saved at: checkpoints/csrnet_epoch1.pth


Epoch 2/100: 100%|██████████| 300/300 [00:41<00:00,  7.30it/s]



🔵 Epoch 2/100
   Avg Epoch Loss      : 1.001716
   Avg Count Error     : 37.74
   ✔ Checkpoint saved at: checkpoints/csrnet_epoch2.pth


Epoch 3/100: 100%|██████████| 300/300 [00:42<00:00,  7.00it/s]


...


🔵 Epoch 98/100
   Avg Epoch Loss      : 0.035048
   Avg Count Error     : 2.78
   ✔ Checkpoint saved at: checkpoints/csrnet_epoch98.pth


Epoch 99/100: 100%|██████████| 300/300 [00:43<00:00,  6.97it/s]



🔵 Epoch 99/100
   Avg Epoch Loss      : 0.037262
   Avg Count Error     : 3.22
   ✔ Checkpoint saved at: checkpoints/csrnet_epoch99.pth


Epoch 100/100: 100%|██████████| 300/300 [00:43<00:00,  6.96it/s]



🔵 Epoch 100/100
   Avg Epoch Loss      : 0.037266
   Avg Count Error     : 2.94
   ✔ Checkpoint saved at: checkpoints/csrnet_epoch100.pth


In [ ]:
from google.colab import files
files.download("checkpoints/csrnet_epoch100.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>